#리스트 슬라이스 테스트

In [ ]:
test_list = [i for i in range(1,20)]
test_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
day = 5
day_save = []
checker = 5
test_list = [1]*23
length = len(test_list)
print(length)
rest = length%day
mok = length//day

start = 0
check=0
while start<length:
  print(test_list[start:start + mok + check])
  day_save.append(test_list[start:start + mok + check])
  start += mok+check
  checker-=1
  if checker<=rest:
    check = 1
print(day_save)

23
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[1, 1, 1, 1, 1]
[[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]


#메소드로 만들기

In [ ]:
def path_divider(day,path):
  path_per_day = []
  checker = day

  length = len(path)
  print('length :',length )

  remainder = length%day
  quotient = length//day

  start = 0
  check=0

  while start<length:
    print(path[start:start + quotient + check])
    path_per_day.append(path[start:start + quotient + check])
    start += quotient+check
    checker-=1
    if checker<=remainder:
      check = 1

  return path_per_day

##메소드 동작 테스트

In [ ]:
day = 5
test_list = [1]*14
result = path_divider(day,test_list)
print("result:",result)

length : 14
[1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
[1, 1, 1]
result: [[1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]


#user_id를 통한 추천코스를 쪼개서 반환해주기

In [ ]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM tourist_attraction WHERE id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  result = theme_score / len(survey_result)
  return result
  
def load_course(day,db):
  cursor= db.cursor()
  min_length = 2*day
  max_length = 4*day
  load_course_sql= f'SELECT id,title,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster,places FROM course_ WHERE length>={min_length} and length<={max_length}'
  cursor.execute(load_course_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['id','title','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster','places']

  return df
def recommend_by_theme(user_id, day):
  top = 5
  db = load_db()
  course = load_course(day,db)
  user_taste = calculate_taste(user_id,db)
  user_taste = user_taste.reshape(1,-1)

  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']

  print(user_taste)
  similarity = cosine_similarity(user_taste, course[tag])
  similarity = similarity[0]
  print(similarity.shape)
      
  print('similarity : ',similarity)
  
  result_index = similarity.argsort()
  result_index = result_index
  
  print("result_index : ",result_index)
#line61~64 확인용
  for i in range(-1,-6,-1):
    now_idx = result_index[i]
    sim = similarity[now_idx]
    print("similarity : ",sim,"df index : ",now_idx)

  top_N_result = result_index[-1:-(top+1):-1]
  print("top_N_result",top_N_result)
  
  #결과는 가장 일치율이 높은거 1개의 index만 던져주자
  result = top_N_result[0]
  result_df = course.iloc[result]
  print(result_df)
  path = path_divider(day,result_df['places'])
  print('path : ',path)
  return path

In [ ]:
user_id = 184
day = 4

result = recommend_by_theme(user_id,day)

[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
place :  32 	now :  (2, 3, 1, 1, 3, 1, 3, 0)
place :  19 	now :  (2, 3, 1, 2, 3, 0, 3, 0)
place :  13 	now :  (2, 3, 1, 3, 3, 1, 3, 0)
place :  6 	now :  (3, 2, 1, 1, 2, 1, 3, 1)
place :  33 	now :  (0, 0, 1, 1, 0, 3, 2, 0)
place :  8 	now :  (2, 3, 2, 3, 2, 2, 3, 0)
place :  10 	now :  (2, 2, 1, 1, 3, 1, 2, 0)
place :  25 	now :  (2, 3, 2, 3, 3, 2, 3, 0)
place :  41 	now :  (2, 3, 1, 1, 1, 2, 3, 0)
place :  72 	now :  (3, 3, 3, 0, 2, 0, 3, 0)
[[2.  2.5 1.4 1.6 2.2 1.3 2.8 0.1]]
(1840,)
similarity :  [0.99016423 0.98707509 0.98717505 ... 0.97452429 0.98222014 0.99112965]
result_index :  [ 116 1291 1066 ... 1238 1257 1101]
similarity :  0.9993330202582142 df index :  1101
similarity :  0.998752727382239 df index :  1257
similarity :  0.9986198964474504 df index :  1238
similarity :  0.9983989925820225 df index :  115
similarity :  0.9983731353255855 df index :  1207
top_N_result [1101 1257 1238  115 1207]
id                                     2263

In [ ]:
result

1101

In [ ]:
mystring = '0'
convert = list(map(int,mystring.split(',')))
convert

[0]